#Task Write-up for an Entry-Level Generative AI Engineer

By: Hammad Ahmad (muhammadhamad913@gmail.com)

#1: Summarizer Task

##Install libraries

In [1]:
!pip install gdown
!pip -q install langchain openai==0.27.0 tiktoken
!pip install PyPDF2
!pip install fpdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256

## Download file/book from provided link

In [2]:
import gdown

# Google Drive file URL
file_url = 'https://drive.google.com/uc?id=1WYaEoSozvUmJ_Lt5clAVs8NuSu43h9ps'

# Output file name
output_file = 'Crime and Punishment.pdf'

# Download file from Google Drive
gdown.download(file_url, output_file, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1WYaEoSozvUmJ_Lt5clAVs8NuSu43h9ps
To: /content/Crime and Punishment.pdf
100%|██████████| 2.49M/2.49M [00:00<00:00, 147MB/s]


'Crime and Punishment.pdf'

## Setting up Summarization Chain



In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [4]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

In [5]:
text_splitter = CharacterTextSplitter()

In [6]:
import PyPDF2

with open('/content/Crime and Punishment.pdf', 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    page_texts = []
    num_pages_to_read = min(20, len(pdf_reader.pages))
    for page_num in range(num_pages_to_read):
        page = pdf_reader.pages[page_num]
        page_texts.append(page.extract_text())

for text in page_texts:
    texts = text_splitter.split_text(text)

In [7]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in page_texts[1:20]]

In [8]:
docs

[Document(page_content='Crime and Punishment \x18Translator’s Preface\nA few words about Dostoevsky himself may help the Eng -\nlish reader to understand his work.\nDostoevsky was the son of a doctor. His parents were \nvery hard- working and deeply religious people, but so poor \nthat they lived with their five children in only two rooms. \nThe father and mother spent their evenings in reading aloud \nto their children, generally from books of a serious charac -\nter.\nThough always sickly and delicate Dostoevsky came out \nthird in the final examination of the Petersburg school of \nEngineering. There he had already begun his first work, \n‘Poor Folk.’\nThis story was published by the poet Nekrassov in his \nreview and was received with acclamations. The shy, un -\nknown youth found himself instantly something of a \ncelebrity. A brilliant and successful career seemed to open \nbefore him, but those hopes were soon dashed. In 1849 he \nwas arrested.\nThough neither by temperament nor

In [9]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [10]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  The translator's preface to "Crime and Punishment" provides background information on the author,
Fyodor Dostoevsky, and his experiences with poverty, imprisonment, and illness. The novel follows
the story of Raskolnikov, a young man who commits a murder and struggles with the psychological
consequences. The passage also describes Raskolnikov's interactions with an old woman, his visit to
a tavern, and his encounter with a retired clerk who appears to be mad.


In [11]:
# for summarizing each part
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [12]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [13]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             verbose=True
                             )


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Crime and Punishment Translator’s Preface
A few words about Dostoevsky himself may help the Eng -
lish reader to understand his work.
Dostoevsky was the son of a doctor. His parents were 
very hard- working and deeply religious people, but so poor 
that they lived with their five children in only two rooms. 
The father and mother spent their evenings in reading aloud 
to their children, generally from books of a serious charac -
ter.
Though always sickly and delicate Dostoevsky came out 
third in the final examination of the Petersburg school of 
Engineering. There he had already begun his first work, 
‘Poor Folk.’
This story was published by the poet Nekrassov in his 
review and was received with acclamations. The shy, un -
known youth found himself instantly something of a 
celebrity. A brilliant and successful career seemed to op

In [14]:
print(wrapped_text)



The translator's preface to "Crime and Punishment" provides background information on the author,
Fyodor Dostoevsky, and his experiences with poverty, imprisonment, and illness. The novel follows
the story of a young man named Raskolnikov who commits a murder and struggles with the psychological
consequences. The passage also describes Raskolnikov's interactions with an old woman, his visit to
a tavern, and his encounter with a retired clerk.


##Export the final Summary in PDF

In [15]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=11)

# Set the left margin
pdf.set_left_margin(10)

# Add the content to the PDF with structured formatting
lines = wrapped_text.split("\n")
for line in lines:
    pdf.write(10, line)  # Adjust the line height (second argument) to add space between lines
    pdf.ln()  # Add a new line after each line

# Save the PDF file
pdf.output("Summary of Crime and Punishment.pdf")


''

By: Hammad Ahmad (muhammadhamad913@gmail.com)